In [95]:
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

import torch

from wbml import plot

# torch.set_printoptions(precision=8, sci_mode=False, edgeitems=5)
torch.set_printoptions(profile='full') # 'short'

colors = sns.color_palette("bright")
sns.set_style("whitegrid")
sns.set_palette(colors)
matplotlib.rcParams["figure.dpi"] = 500  # for high quality, retina plots
matplotlib.use("Agg")


In [56]:
path = "results/07-19-16.29.49_seq-pvi-1c-1g-2000l-40n-40m-verify-old-elbo-linspace/model/server_all_preds_iter_0.csv"

In [97]:
wide_prior = "results/07-19-17.41.28_seq-mfvi-1c-1g-2000l-40n-40m-neal-random-mu/model/_vs.pt"
three_global = "results/07-19-21.05.10_seq-mfvi-1c-3g-2000l-40n-40m-positive-nz-own-elbo/model/_vs.pt"
last = "results/07-19-20.42.55_seq-mfvi-1c-1g-2000l-40n-40m-positive-nz/model/_vs.pt"
vs_path = last


In [98]:
main_dir = os.path.dirname(os.path.abspath(os.getcwd()))
vs_state_dir = torch.load(os.path.join(main_dir, vs_path))

In [103]:
vs_state_dir.keys()

dict_keys(['ts.client0_layer0_yz', 'ts.client0_layer0_nz', 'ts.client0_layer1_yz', 'ts.client0_layer1_nz', 'ts.client0_layer2_yz', 'ts.client0_layer2_nz'])

In [105]:
vs_state_dir[list(vs_state_dir.keys())[-1]]

tensor([[  22.8519,   21.1552,  247.7902,  102.8024,  518.8912,  167.7104,
           95.0006,  101.1505,   91.5170,  375.7248,   90.8197,  214.4642,
          131.4912,  436.6009,  358.9222,  254.1986,  121.1544,  239.8508,
          182.2157,   80.1596,  255.6025,   74.9252,  101.9205,  132.6635,
           55.8954,  109.1249,   41.9648,   60.1175,  123.2675,  129.5191,
          332.3310,  332.5177,   72.0024,  184.1632,  250.7320,   48.0675,
          237.3905,   60.5434,  121.7616,  349.8449,   34.5149,   21.9797,
          385.5743,  101.5507,  247.4278,  347.5464,   92.6308,   57.4028,
           78.0607,  135.9774, 2355.5796]], requires_grad=True)

In [100]:
vs_state_dir[list(vs_state_dir.keys())[1]].T

tensor([[20.5925, 34.1679, 27.4666, 21.2407, 20.2142, 27.9008, 20.7800, 27.8622,
         22.1931, 17.4695, 12.6132, 32.3998, 17.0270, 14.3238, 11.9937, 11.5298,
         16.8873, 25.4217, 29.9991, 36.8027, 12.0405, 27.6854, 12.6119, 19.3252,
         46.1261, 26.4788, 19.2592, 14.9164, 16.7948, 15.5978, 26.3602, 15.5579,
         26.2386, 22.2683, 20.9826, 18.0023, 22.7819, 14.4615, 20.6489, 22.7406,
         30.8119, 26.2076, 18.5041, 24.7834, 19.9679, 22.2622, 15.7326, 15.1691,
         27.1299, 33.6218],
        [ 3.9260,  3.9688,  4.1291,  8.3202,  3.9094,  4.0506,  4.7102,  4.3542,
          5.8102,  2.2888,  4.1537,  2.9412,  4.0309,  4.0741,  4.0453,  7.6046,
          6.3933,  4.1721,  3.9041,  4.3713,  5.4350,  5.6949,  6.3666,  7.7233,
          3.6836,  4.4715,  4.5683,  5.5955,  6.5735, 14.2698,  5.8694,  4.2334,
          4.9837,  4.8707,  9.4251, 12.2005,  6.9313,  7.0257,  3.4412,  3.2987,
          5.3678,  7.7373,  9.0401,  3.1522,  4.7592,  5.1386,  3.4020,  4.0592,


In [112]:
for k, v in vs_state_dir.items():
    print(k, v.shape)
    if len(v.shape) == 3:
        print(v[..., 0])
    else:
        print(v)

ts.client0_layer0_yz torch.Size([50, 2, 1])
tensor([[ -6.3697,  -4.0143],
        [  0.3970,  -3.3238],
        [  1.0216,  -2.9505],
        [ -7.9041,  -4.4183],
        [ -1.7430,  -2.7450],
        [ -5.5317,  -4.2161],
        [  2.4881,  -3.5989],
        [  6.6529,  -4.9439],
        [ -7.3312,  -4.7131],
        [  2.6698,  -3.0309],
        [ -4.7030,  -2.7804],
        [ -4.6135,  -3.5988],
        [  9.3431,  -5.9034],
        [  1.5005,  -2.7304],
        [ 12.2503,  -6.3596],
        [  8.0462,  -4.8320],
        [ -7.1720,  -4.2065],
        [ -3.5391,  -3.4485],
        [ -3.2883,  -3.3829],
        [ -0.7128,  -2.7182],
        [ -1.5950,  -2.8069],
        [ -7.4469,  -4.2053],
        [ 10.8146,  -6.7076],
        [ -9.8929,  -4.8605],
        [ -6.4673,  -3.8130],
        [  5.1734,  -3.8411],
        [ -4.4212,  -3.0612],
        [  7.7890,  -5.0816],
        [ -7.5279,  -4.3258],
        [  9.8321,  -1.3865],
        [ -7.8813,  -4.6952],
        [ 10.4373,  -5.746